In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import time
from googletrans import Translator



# Load the model
model_dict = pickle.load(open('model.p', 'rb'))
model = model_dict['model']

# Open the camera
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Define labels dictionary
labels_dict = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
    10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S',
    19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: ' '
}

# Initialize variables
predicted_letters = ""
start_time = None
previous_letter = None

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read frame from the camera.")
        break

    # Process the frame using MediaPipe Hands
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            data_aux = []
            x_ = []
            y_ = []

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

            x1 = int(min(x_) * frame.shape[1]) - 10
            y1 = int(min(y_) * frame.shape[0]) - 10
            x2 = int(max(x_) * frame.shape[1]) - 10
            y2 = int(max(y_) * frame.shape[0]) - 10

            prediction = model.predict([np.asarray(data_aux)])
            predicted_character = labels_dict[int(prediction[0])]

            if predicted_character == previous_letter:
                # If the same letter is predicted, check if it has been shown for at least 2 seconds
                if start_time is not None and time.time() - start_time >= 2:
                    predicted_letters += predicted_character
                    start_time = time.time()  # Reset the start time
            else:
                # If a new letter is predicted, update previous_letter and start_time
                previous_letter = predicted_character
                start_time = time.time()

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
            cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                        cv2.LINE_AA)

            # Display predicted letters on the frame
            cv2.putText(frame, "Predicted Letters: " + predicted_letters, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [3]:
from translate import Translator
from gtts import gTTS
import pygame
import tempfile
import os


# Create a Translator object
translator = Translator(to_lang="hi")  # "hi" is the language code for Hindi

# Perform the translation
hindi_text = translator.translate(predicted_letters)

# Create a gTTS object for Hindi text
tts = gTTS(text=hindi_text, lang='hi', slow=False)

# Save the speech to a temporary file with a modified name
temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
temp_file_name = temp_file.name.replace("_", "-")  # Replace underscores with hyphens
temp_file.close()

# Save the speech to the modified temporary file name
tts.save(temp_file_name)

# Play the speech using pygame
pygame.mixer.init()
pygame.mixer.music.load(temp_file_name)
pygame.mixer.music.play()

# Wait for the speech to finish playing
while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)

# Remove the temporary file after playing
os.remove(temp_file_name)


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\ACER\\AppData\\Local\\Temp\\tmpb3y-cmlu.mp3'